In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%run '/content/drive/My Drive/Miniproject_5sem/model.ipynb' import NeuralNet

Mounted at /content/drive


In [ ]:
import random
import json
import torch
import numpy as np


In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer 

def tokenize(sentence):
  return nltk.word_tokenize(sentence)

stemmer = PorterStemmer()
def stem(word):
  return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
  tokenized_sentence = [stem(w) for w in tokenized_sentence]

  bag = np.zeros(len(all_words))
  for idx, w, in enumerate(all_words):
    if w in tokenized_sentence:
      bag[idx] = 1
  
  return bag

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import files

uploaded = files.upload()
intents = json.loads(uploaded['intents.json'].decode("utf-8"))

Saving intents.json to intents.json


In [ ]:
FILE = "/content/drive/My Drive/Miniproject_5sem/data.pth"
data = torch.load(FILE)


In [ ]:
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

In [ ]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

NeuralNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [ ]:
agent_name = "Bot"
print("Let's chat! type 'quit' to exit")

while(True):
  sentence = input("You: ")
  if sentence == "quit":
        break
  sentence = tokenize(sentence)
  X = bag_of_words(sentence, all_words)
  X = X.reshape(1, X.shape[0])
  X = torch.from_numpy(X).to(device) #convert nump array to tensor on gpu

  output = model(X.float())
  _, predicted = torch.max(output, dim=1)
  tag = tags[predicted.item()]

  probs = torch.softmax(output, dim=1)
  prob = probs[0][predicted.item()]

  if prob.item() > 0.50:
    for intent in intents["intents"]:
      if tag == intent["tag"]:
        print(f"{agent_name}: {random.choice(intent['responses'])}")
  else:
    print(f"{agent_name}: I do not understand...")

  

   


Let's chat! type 'quit' to exit
Bot: We sell Burger and pizza
Bot: We accept most major credit cards, and Paypal
